In [1]:
# This script calls Rscript to run cluster_completeness_testing.R
import os
import pandas as pd

outdir = '../result/completeness'
cluster_df = pd.read_csv('../result/GCN_fix_tree/leaves_cluster.tsv', sep='\t', index_col=0, header=0)
cluster_df.fillna('NA', inplace=True)


In [ ]:
# extract all superclusters' completeness
sclusters = set(cluster_df['supercluster'])
super_d = pd.DataFrame(columns=['scluster', 'sp'])
for sp in cluster_df.index:
    if cluster_df.loc[sp, 'supercluster'] == 'NA':
        continue
    super_d.loc[super_d.shape[0], ] = [cluster_df.loc[sp, 'supercluster'], sp.replace('-', '_')]
super_d.to_csv(os.path.join(outdir, 'super_species.tsv'), sep='\t', index=False, header=False)

In [ ]:
# extract all subclusters' completeness under each supercluster
completeness_d = pd.read_csv(os.path.join(outdir, 'genome_module.completeness.tsv'), sep='\t', header=0, index_col=0)
for scluter in sclusters:
    species = cluster_df[cluster_df['supercluster'] == scluter].index
    opath = os.path.join(outdir, scluter + '.cluster_species.tsv')
    c_df = pd.DataFrame(columns=['cluster', 'sp'])
    for sp in cluster_df[cluster_df['supercluster'] == scluter].index:
        c_df.loc[c_df.shape[0], ] = [cluster_df.loc[sp, 'cluster'].replace('-', '_'), sp.replace('-', '_')]
    sp_list = list(c_df['sp'])
    part_completeness = completeness_d.loc[sp_list, :]
    part_completeness.to_csv(os.path.join(outdir, scluter + '.genome_module.completeness.tsv'), sep='\t', index=True, header=True)
    c_df.to_csv(opath, sep='\t', index=False, header=False)
    

In [4]:
# call Rscript for supercluster completeness testing
cmd = 'Rscript cluster_completeness_testing.R genome_module.completeness.tsv super_species.tsv super_cluster.module_comp.wilcox.testing.tsv {}'.format(outdir)
os.system(cmd)

0

In [5]:
# call Rscript for supercluster completeness testing
for scluster in sclusters:
    cmd = 'Rscript cluster_completeness_testing.R {}.genome_module.completeness.tsv {}.cluster_species.tsv {}.cluster.module_comp.wilcox.testing.tsv {}'.format(scluster, scluster, scluster, outdir)
    os.system(cmd)
